In [15]:
################################################
# Importando las librerias
################################################
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cv
from geopy import distance # Biblioteca para calculos geograficos

import random
import folium
from folium import PolyLine
import matplotlib

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Read the GTFS files
stops = pd.read_csv('/content/drive/MyDrive/Code/Data/stops.txt', delimiter=',')
stop_times = pd.read_csv('/content/drive/MyDrive/Code/Data/stop_times.txt', delimiter=',')
trips = pd.read_csv('/content/drive/MyDrive/Code/Data/trips.txt', delimiter=',')
routes = pd.read_csv('/content/drive/MyDrive/Code/Data/routes.txt', delimiter=',')

In [17]:
# INPUTS
m_buses = 3 # number of available buses
#selec_trip_id = 32394  # 23 pts
#selec_trip_id =  32566 # 8 pts
selec_trip_id =  32402 # 13 pts -> just as an example, select the set of stops
# using one of the the trip_ids of the Zubieta file

# Merge trips and routes to have the name and the trip_id together
routes = routes[["route_id","route_long_name"]]
trips = trips[["route_id","trip_id"]]
trips = pd.merge(trips, routes, on='route_id', how='left')
trips = trips.drop(columns='route_id')

# Merge stop_times and stops to have the trip_id with the stop latitude and longitud
stops = stops[["stop_id","stop_lat","stop_lon"]]
stop_times = stop_times[["trip_id","stop_id",]]
stops_coord = pd.merge(stop_times, stops, on='stop_id', how='left')
stops_coord = stops_coord.drop(columns='stop_id')

# Merge the two DataFrames
df = pd.merge(stops_coord, trips, on='trip_id', how='left')

In [18]:
df

,trip_id,stop_lat,stop_lon,route_long_name
0,32394,43.271070,-2.033995,Zubieta - Donostia
1,32394,43.271635,-2.032785,Zubieta - Donostia
2,32394,43.267618,-2.023838,Zubieta - Donostia
3,32394,43.269221,-2.021350,Zubieta - Donostia
4,32394,43.266088,-2.019592,Zubieta - Donostia
...,...,...,...,...
9361,32760,43.307991,-1.979479,Asteasu-Donostia Por Amara
9362,32760,43.313709,-1.981575,Asteasu-Donostia Por Amara
9363,32760,43.317394,-1.981963,Asteasu-Donostia Por Amara
9364,32760,43.320067,-1.979942,Asteasu-Donostia Por Amara


In [19]:
Zubieta_routes = df[df['route_long_name'].str.contains('Zubieta', case=False)] # El parámetro case=False ignora mayúsculas y minúsculas
print("Zubieta routes:")
Zubieta_routes

Zubieta routes:


,trip_id,stop_lat,stop_lon,route_long_name
0,32394,43.271070,-2.033995,Zubieta - Donostia
1,32394,43.271635,-2.032785,Zubieta - Donostia
2,32394,43.267618,-2.023838,Zubieta - Donostia
3,32394,43.269221,-2.021350,Zubieta - Donostia
4,32394,43.266088,-2.019592,Zubieta - Donostia
...,...,...,...,...
5148,32610,43.266154,-2.019442,Lasarte-Zubieta
5149,32610,43.269605,-2.019970,Lasarte-Zubieta
5150,32610,43.269266,-2.021513,Lasarte-Zubieta
5151,32610,43.267663,-2.024042,Lasarte-Zubieta


In [20]:
# Iterar a través de los trip_id únicos y guardar el trip seleccionado
for trip_id in Zubieta_routes['trip_id'].unique():
    trip_df = Zubieta_routes[Zubieta_routes['trip_id'] == trip_id]
    print(trip_id, len(trip_df))
    if trip_id ==  selec_trip_id:
       puntos = list(zip(trip_df['stop_lat'], trip_df['stop_lon']))
print(puntos)

"""
################################################
# Data original
################################################
puntos = [(-12.059296, -76.975893),
          (-12.079575, -77.009686),
          (-12.087303, -76.996620),
          (-12.084391, -76.975651),
          (-12.063603, -76.960483),
          (-12.056762, -77.014452),
          (-12.011531, -77.002383)]
"""

32394 23
32395 23
32396 6
32397 6
32398 6
32399 6
32400 6
32401 6
32402 13
32403 13
32404 18
32518 24
32519 24
32566 8
32567 8
32568 8
32580 8
32581 8
32582 8
32583 8
32584 8
32585 8
32604 6
32605 6
32606 6
32607 6
32608 6
32609 6
32610 6
[(43.2710697672, -2.03399518119), (43.2716349794, -2.03278535665), (43.2676178598, -2.02383842655), (43.2692209222, -2.02134997579), (43.2660882619, -2.01959219703), (43.2614048798, -2.01783341168), (43.2568468416, -2.01858930655), (43.253512211, -2.01944403314), (43.2538993572, -2.02267626405), (43.2460101566, -2.02533781575), (43.2285398957, -2.02666531662), (43.2228212602, -2.02571141907), (43.2095088069, -2.0298682003)]


'\n################################################\n# Data original\n################################################\npuntos = [(-12.059296, -76.975893),\n          (-12.079575, -77.009686),\n          (-12.087303, -76.996620),\n          (-12.084391, -76.975651),\n          (-12.063603, -76.960483),\n          (-12.056762, -77.014452),\n          (-12.011531, -77.002383)]\n'

In [21]:
################################################
# Calculando la matriz de distancias
################################################
n = len(puntos)
C = np.zeros((n,n))

for i in range(0, n):
    for j in range(0, len(puntos)):
        C[i,j] = distance.distance(puntos[i], puntos[j]).km

# Mostrando la matriz de distancias
print('La matriz de distancia es:\n')
print(np.round(C,4))

La matriz de distancia es:

[[0.     0.1166 0.9094 1.0469 1.2937 1.6955 2.0153 2.2805 2.1174 2.8714
  4.7623 5.4023 6.8475]
 [0.1166 0.     0.8525 0.9663 1.2357 1.6629 2.0069 2.2863 2.1345 2.9104
  4.8135 5.4534 6.9061]
 [0.9094 0.8525 0.     0.2693 0.3843 0.8451 1.2703 1.6072 1.527  2.4037
  4.3475 4.9791 6.4743]
 [1.0469 0.9663 0.2693 0.     0.3762 0.9141 1.3929 1.7521 1.7056 2.5989
  4.5401 5.167  6.6698]
 [1.2937 1.2357 0.3843 0.3762 0.     0.5396 1.0299 1.3972 1.3771 2.2789
  4.2109 4.8325 6.341 ]
 [1.6955 1.6629 0.8451 0.9141 0.5396 0.     0.5101 0.8866 0.9219 1.8156
  3.721  4.334  5.8478]
 [2.0153 2.0069 1.2703 1.3929 1.0299 0.5101 0.     0.3769 0.4662 1.3228
  3.2125 3.8242 5.3383]
 [2.2805 2.2863 1.6072 1.7521 1.3972 0.8866 0.3769 0.     0.266  0.9611
  2.8357 3.4475 4.9615]
 [2.1174 2.1345 1.527  1.7056 1.3771 0.9219 0.4662 0.266  0.     0.9027
  2.8359 3.4615 4.9662]
 [2.8714 2.9104 2.4037 2.5989 2.2789 1.8156 1.3228 0.9611 0.9027 0.
  1.9439 2.5764 4.0719]
 [4.7623 4.8135 

In [22]:
################################################
# Resolviendo el Problema de programación entera
################################################

# Definiendo las variables
X = cv.Variable(C.shape, boolean=True)
u = cv.Variable(n, integer=True)
ones = np.ones((n,1))

In [23]:
ones

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [24]:
# Definiendo la función bojetivo
objective = cv.Minimize(cv.sum(cv.multiply(C, X)))

In [25]:
# Definiendo las restricciones
constraints = []
constraints += [X[0,:] @ ones == m_buses]
constraints += [X[:,0] @ ones == m_buses]
constraints += [X[1:,:] @ ones == 1]
constraints += [X[:,1:].T @ ones == 1]
constraints += [cv.diag(X) == 0]
constraints += [u[1:] >= 2]
constraints += [u[1:] <= n]
constraints += [u[0] == 1]

In [26]:
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            constraints += [ u[i] - u[j] + 1  <= (n - 1) * (1 - X[i, j]) ]

In [27]:
# Resolviendo el problema
prob = cv.Problem(objective, constraints)
prob.solve(verbose=False)

16.960384244609408

In [14]:
# Transformando la respuesta en una ruta ordenada
X_sol = np.argwhere(X.value==1)
print('Routes:')
print(X_sol)

ruta = {}
for i in range(0, m_buses):
    ruta['Bus_' + str(i+1)] = [0]
    j = i
    a = 10e10
    while a != 0:
        a = X_sol[j,1]
        ruta['Bus_' + str(i+1)].append(a)
        j = np.where(X_sol[:,0] == a)
        j = j[0][0]
        a = j

# Mostrando la ruta
for i in ruta.keys():
    print('El orden de la ruta del ' + i + ' es:\n')
    print( ' => '.join(map(str, ruta[i])))
    print('')

# Crear un mapa
m = folium.Map(location=[Zubieta_routes['stop_lat'].mean(), Zubieta_routes['stop_lon'].mean()], zoom_start=6)

cols = np.linspace(0,150,m_buses)
cols = [round(cols[i]) for i in range(len(cols))]
r = cols
g = cols
b = cols

cont=0
for i in ruta.keys():
    pts = ruta[i]
    print(ruta[i])
    coordinates = [puntos[j] for j in ruta[i]]
    print(coordinates)
    # Agregar una polilínea que une los puntos
    col = '#%02x%02x%02x' % (r[cont], g[cont], b[cont])
    print(col)
    line = PolyLine(coordinates, color=col, weight=4, opacity=0.8)
    line.add_to(m)
    cont+=1

m.save('/content/drive/MyDrive/Code/Results/mapa.html')

Routes:
[[ 0  1]
 [ 0  2]
 [ 0  8]
 [ 1  0]
 [ 2  0]
 [ 3  0]
 [ 4  3]
 [ 5  4]
 [ 6  5]
 [ 7  6]
 [ 8  9]
 [ 9 10]
 [10 12]
 [11  7]
 [12 11]]
El orden de la ruta del Bus_1 es:

0 => 1 => 0

El orden de la ruta del Bus_2 es:

0 => 2 => 0

El orden de la ruta del Bus_3 es:

0 => 8 => 9 => 10 => 12 => 11 => 7 => 6 => 5 => 4 => 3 => 0

[0, 1, 0]
[(43.2710697672, -2.03399518119), (43.2716349794, -2.03278535665), (43.2710697672, -2.03399518119)]
#000000
[0, 2, 0]
[(43.2710697672, -2.03399518119), (43.2676178598, -2.02383842655), (43.2710697672, -2.03399518119)]
#4b4b4b
[0, 8, 9, 10, 12, 11, 7, 6, 5, 4, 3, 0]
[(43.2710697672, -2.03399518119), (43.2538993572, -2.02267626405), (43.2460101566, -2.02533781575), (43.2285398957, -2.02666531662), (43.2095088069, -2.0298682003), (43.2228212602, -2.02571141907), (43.253512211, -2.01944403314), (43.2568468416, -2.01858930655), (43.2614048798, -2.01783341168), (43.2660882619, -2.01959219703), (43.2692209222, -2.02134997579), (43.2710697672, -2.0339951